In [1]:
import pyarrow.parquet as pq
import torch
import random
import time
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
temp = pq.read_table('./Data/cleaned_taxi_data.parquet')
temp = temp.to_pandas()

trip_info = torch.tensor(temp.values, dtype = torch.float32)

In [9]:
temp.describe()

,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,...,store_and_fwd_flag_0,store_and_fwd_flag_1,payment_type_1,payment_type_2,payment_type_3,payment_type_4,tip_bin_0,tip_bin_1,tip_bin_2,tip_bin_3
count,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,...,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06,2.276461e+06
mean,1.472616e+00,2.017087e+00,9.898870e+00,1.023657e+00,4.994395e-01,1.966696e+00,4.196900e-03,2.999851e-01,1.543090e+01,2.434673e+00,...,9.922112e-01,7.788844e-03,7.807847e-01,2.155697e-01,2.394945e-03,1.250625e-03,3.378955e-01,5.871385e-01,7.437246e-02,5.934650e-04
std,1.039033e+00,1.671309e+00,5.098086e+00,1.194539e+00,1.673157e-02,1.719994e+00,1.745916e-01,2.113585e-03,6.083246e+00,3.988098e-01,...,8.791008e-02,8.791008e-02,4.137149e-01,4.112171e-01,4.887955e-02,3.534207e-02,4.729929e-01,4.923484e-01,2.623761e-01,2.435391e-02
min,1.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e-01,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+00,1.020000e+00,6.500000e+00,0.000000e+00,5.000000e-01,8.000000e-01,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00,...,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.610000e+00,8.500000e+00,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,3.000000e-01,1.416000e+01,2.500000e+00,...,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,2.000000e+00,2.520000e+00,1.200000e+01,2.500000e+00,5.000000e-01,2.860000e+00,0.000000e+00,3.000000e-01,1.790000e+01,2.500000e+00,...,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,7.000000e+00,9.990000e+01,3.000000e+02,4.550000e+01,5.000000e-01,3.500000e+02,2.620000e+01,3.000000e-01,3.573000e+02,2.750000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [3]:
# Base values
RANDOM_SEED = 123
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 64
NUM_EPOCHS = 50

In [4]:
# Normalize and Split Input Data (Train = 75% (1,707,346), Validation = 20% (455,292), Test = 5% (113,823))
torch.manual_seed(RANDOM_SEED)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(trip_info,  [0.75, 0.2, 0.05])

trip_inf_loc = list(range(0, 5)) + list(range(6, 172))
tip_inf_loc = [172, 173, 174, 175]
x = train_dataset.dataset[:, trip_inf_loc]
y = train_dataset.dataset[:, tip_inf_loc]


In [6]:
print(x.shape)
print(y.shape)

torch.Size([2276461, 109])
torch.Size([2276461, 67])


In [7]:

class MLP(torch.nn.Module):

    def __init__(self, num_features, num_hidden_1, num_classes):
        super(MLP, self).__init__()
        
        self.num_classes = num_classes
        
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.bn1 = torch.nn.BatchNorm1d(self.linear_1.weight.size(0))
        self.linear_out = torch.nn.Linear(num_hidden_1, num_classes)
        
    def forward(self, x):
         
        out = self.linear_1(x)
        print(out.shape)
        print(self.linear_1.weight.size(0))
        out = self.bn1(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas
    
#################################
### Model Initialization
#################################

random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

model = MLP(num_features=109,
            num_hidden_1=100,
            num_classes=67)


model = model.to(DEVICE)


In [10]:

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for epoch in range(NUM_EPOCHS):
    
    model.train()
    # pytorch is expecting minibatch of examples
    for i in range(y.size()[0]):
        
        ### PREPARE MINIBATCH
        features = x[i].view(-1, 109).to(DEVICE)
        targets = y[i].to(DEVICE) # This is just giving one example of y
        # We want it to be of size (batch size) x (# of classes)
        # Check examples of FashionMNIST dataset
            
        ### FORWARD AND BACK PROP
        logits, probas = model.forward(features)
        print(f'Target: {targets}')
        print(f'Logits: {logits}')
        print(f'Probabilities: {probas}')
        cost = F.cross_entropy(torch.flatten(logits), targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ## LOGGING
        if not i % 20:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {i:03d}/{len(x):03d} |' 
                   f' Cost: {cost:.4f}')

    # no need to build the computation graph for backprop when computing accuracy
    model.eval()
    

torch.Size([1, 100])
100


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 100])